In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.cassandra._
val sparkSession = SparkSession.builder().config(sc.getConf).getOrCreate()
import sparkSession.implicits._

In [4]:
sparkSession.read.cassandraFormat(keyspace = "yelp", table = "review").load().createOrReplaceTempView("review")
sparkSession.read.cassandraFormat(keyspace = "yelp", table = "business").load().createOrReplaceTempView("business")

In [79]:
sparkSession.sql("""
    select
        business_id,
        name business_name,
        bad_thing_count,
        review_id review_happened_at,
        date
    from(
        select
            review_id,
            business_id,
            name,
            date,
            bad_thing_count,
            case
                when
                     stars<=3 and
                     bad_thing_count>=3
                 then 1
                 else 0
                 end
                 as bad_thing_happened        
        from(
            select
                review_id,
                business_id,
                name,
                stars,
                date,
                sum(bad_thing_lasting) over 
                         (PARTITION BY business_id ORDER BY date ROWS BETWEEN 5 PRECEDING AND CURRENT ROW) bad_thing_count
            from(
                select
                    r.review_id,
                    b.business_id,
                    b.name,
                    r.stars,
                    r.date,
                    case
                        when
                            lag(r.stars) over (PARTITION BY b.business_id ORDER BY r.date)<=3
                        then 1 
                        else 0 
                        end
                        as bad_thing_lasting
                from business b
                    inner join review r on r.business_id = b.business_id
            ) T
        )
    )
    where bad_thing_happened=1
    order by date
"""
).show(50,false)

+----------------------+-------------+---------------+----------------------+----------+
|business_id           |business_name|bad_thing_count|review_happened_at    |date      |
+----------------------+-------------+---------------+----------------------+----------+
|EDqCEAGXVGCH4FJXgqtjqg|Pizza Pizza  |3              |Mz0vJLJTTRwyYKQlYOEHCA|2013-05-23|
|EDqCEAGXVGCH4FJXgqtjqg|Pizza Pizza  |4              |nwpS4voFjschTD8mref9Yg|2014-05-30|
|cdk-qqJ71q6P7TJTww_DSA|Comfort Inn  |3              |BAtxh2r4VngJ4RXLSIlZWQ|2014-06-27|
|EDqCEAGXVGCH4FJXgqtjqg|Pizza Pizza  |5              |8yrO1JkClTjS8_8oCS0AQg|2014-08-24|
|cdk-qqJ71q6P7TJTww_DSA|Comfort Inn  |4              |QSftxVxeF_PenhB9ndQOEw|2015-03-01|
|cdk-qqJ71q6P7TJTww_DSA|Comfort Inn  |5              |tQysCIGKX3g5QPtIeeuvSA|2015-11-29|
|cdk-qqJ71q6P7TJTww_DSA|Comfort Inn  |5              |u1HWRi-AxbyeDFZWRzBwVA|2016-05-15|
+----------------------+-------------+---------------+----------------------+----------+

